# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
import helper

# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [2]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [3]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [4]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    sequences = tokenizer.texts_to_sequences(x)
    return sequences, tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

Using TensorFlow backend.


{'brown': 4, 'jumps': 6, 'jove': 11, 'quick': 2, 'sentence': 21, 'lexicography': 15, 'over': 7, 'a': 3, 'prize': 17, 'the': 1, 'lazy': 8, 'short': 20, 'by': 10, 'is': 19, 'this': 18, 'study': 13, 'fox': 5, 'of': 14, 'won': 16, 'dog': 9, 'my': 12}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [5]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post', truncating='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [6]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [7]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Simple function to print and compare translations

In [9]:
def print_predictions(model, X, y, tokenizer, number_of_predictions=10):
    '''
    This function is used to print predictions made by the model followed
    by the correct translation
    
    :param model: is the model that will make the predictions
    :param X: preprocessed and padded inputs
    :param y: correct sentences
    :param number_of_predictions: number of predictions to be printed
    '''
    prediction_separator = '-------------------------------------------------------'
    
    for idx in range(number_of_predictions):
        print(prediction_separator)
        print()
        print('Predicted sentence:\n{0}'.format(logits_to_text(model.predict(X[idx:idx+1])[0], tokenizer)))
        print()
        print('Correct translation:\n{0}'.format(y[idx:idx+1][0]))

### Difining files weights and models files

In [10]:
# Simple RNN files
simple_rnn_best_weights = 'simple_rnn_best_weights.hdf5'
simple_rnn_file = 'simple_rnn.hdf5'

# Embed RNN Files
embed_rnn_best_weights = 'embed_rnn_best_weights.hdf5'
embed_rnn_file = 'embed_rnn.hdf5'

# Bidirectional RNN Files
bd_rnn_best_weights = 'db_rnn_best_weights.hdf5'
bd_rnn_file = 'bd_rnn.hdf5'

# Autoencoder RNN Files
encdec_rnn_best_weights = 'autoenc_rnn_best_weights.hdf5'
encdec_rnn_file = 'autoenc_rnn.hdf5'

# Final RNN Model Files
final_rnn_best_weights = 'final_rnn_best_weights.hdf5'
final_rnn_file = 'final_rnn.hdf5'

### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [10]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    
    learning_rate = 0.01
    
    inputs = Input(shape=input_shape[1:])
    gru0 = GRU(14, return_sequences=True, dropout=0.1)(inputs)
    gru1 = GRU(28, return_sequences=True, dropout=0.2)(gru0)
    gru2 = GRU(56, return_sequences=True, dropout=0.4)(gru1)
    dense0 = TimeDistributed(Dense(english_vocab_size + french_vocab_size, activation='tanh'))(gru2)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dense0)
    model = Model(inputs=inputs, outputs=logits)

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1, # Adding 1 to avoid tokenizer error
    len(french_tokenizer.word_index) + 1)  # Adding 1 to the other tokenized vector

simple_rnn_model.summary()

checkpointer = ModelCheckpoint(filepath=simple_rnn_best_weights, save_best_only=True, verbose=1)
simple_rnn_model.fit(tmp_x, preproc_french_sentences,
                     batch_size=1024, epochs=60,
                     validation_split=0.2, shuffle=True,
                     callbacks=[checkpointer])

print()
print('Loading best weights...')
simple_rnn_model.load_weights(simple_rnn_best_weights)

print('Saving model for later use...')
simple_rnn_model.save(simple_rnn_file)

print()

# Print prediction(s)
print_predictions(model=simple_rnn_model, X=tmp_x,
                  y=french_sentences, tokenizer=french_tokenizer, number_of_predictions=5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_4 (GRU)                  (None, 21, 14)            672       
_________________________________________________________________
gru_5 (GRU)                  (None, 21, 28)            3612      
_________________________________________________________________
gru_6 (GRU)                  (None, 21, 56)            14280     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 545)           31065     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 345)           188370    
Total params: 237,999
Trainable params: 237,999
Non-trainable params: 0
_________________________________________________________________
Trai

110288/110288 [==============================] - 14s - loss: 0.8328 - acc: 0.7194 - val_loss: 2.3551 - val_acc: 0.5578
Epoch 60/60
110288/110288 [==============================] - 14s - loss: 0.8315 - acc: 0.7193 - val_loss: 2.3189 - val_acc: 0.5646

Loading best weights...
Saving model for later use...

Predicted sentence: new jersey est parfois chaud en l' mais il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct translation: ["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]


### This can be used to load the model

In [10]:
# Loading the model
simple_rnn_model = load_model(simple_rnn_file)

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[1], 1))

# Print prediction(s)
print_predictions(model=simple_rnn_model, X=tmp_x,
                  y=french_sentences, tokenizer=french_tokenizer, number_of_predictions=5)

-------------------------------------------------------

Predicted sentence:
new jersey est parfois chaud en l' mais il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct translation:
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
-------------------------------------------------------

Predicted sentence:
les états unis est parfois chaud en l' mais il est parfois parfois en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct translation:
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
-------------------------------------------------------

Predicted sentence:
paris est parfois parfois en en et il il parfois parfois en à <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct translation:
california est généralement calme en mars , et il est généralement chaud en juin .
-------------------------------------------------------

Predicted sentence:
les états unis est parfois chaud en printemps et 

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [12]:
from keras.layers.embeddings import Embedding


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate = 0.01
     
    inputs = Input(shape=input_shape[1:])
    embed = Embedding(english_vocab_size, int(input_shape[1]/3))(inputs)
    gru0 = GRU(14, return_sequences=True, dropout=0.1)(embed)
    gru1 = GRU(28, return_sequences=True, dropout=0.2)(gru0)
    gru2 = GRU(56, return_sequences=True, dropout=0.4)(gru1)
    dense0 = TimeDistributed(Dense(english_vocab_size + french_vocab_size, activation='tanh'))(gru2)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dense0)
    model = Model(inputs=inputs, outputs=logits)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_embed_model(embed_model)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[1]))

# TODO: Train the neural network
embed_rnn_model = embed_model(input_shape=tmp_x.shape,
                              output_sequence_length=preproc_french_sentences[1],
                              english_vocab_size=len(english_tokenizer.word_index) + 1,
                              french_vocab_size=len(french_tokenizer.word_index) + 1)
embed_rnn_model.summary()

checkpointer =  ModelCheckpoint(filepath=embed_rnn_best_weights, save_best_only=True, verbose=1)

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=512,
                    epochs=100, validation_split=0.2,
                    callbacks=[checkpointer])

print()
print('Loading best weights')
embed_rnn_model.load_weights(embed_rnn_best_weights)
print('Saving embed model for later usage.')
embed_rnn_model.save(embed_rnn_file)

# TODO: Print prediction(s)
print_predictions(model=embed_rnn_model, X=tmp_x,
                  y=french_sentences, tokenizer=french_tokenizer, number_of_predictions=5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 21, 7)             1400      
_________________________________________________________________
gru_10 (GRU)                 (None, 21, 14)            924       
_________________________________________________________________
gru_11 (GRU)                 (None, 21, 28)            3612      
_________________________________________________________________
gru_12 (GRU)                 (None, 21, 56)            14280     
_________________________________________________________________
time_distributed_7 (TimeDist (None, 21, 545)           31065     
_________________________________________________________________
time_distributed_8 (TimeDist (None, 21, 345)           188370    
Total para

110288/110288 [==============================] - 22s - loss: 0.2821 - acc: 0.9056 - val_loss: 0.2581 - val_acc: 0.9132
Epoch 54/100
110288/110288 [==============================] - 22s - loss: 0.2871 - acc: 0.9044 - val_loss: 0.2630 - val_acc: 0.9103
Epoch 55/100
110288/110288 [==============================] - 22s - loss: 0.2816 - acc: 0.9059 - val_loss: 0.2494 - val_acc: 0.9152
Epoch 56/100
110288/110288 [==============================] - 22s - loss: 0.2928 - acc: 0.9029 - val_loss: 0.2481 - val_acc: 0.9165
Epoch 57/100
110288/110288 [==============================] - 22s - loss: 0.2807 - acc: 0.9058 - val_loss: 0.2487 - val_acc: 0.9165
Epoch 58/100
110288/110288 [==============================] - 22s - loss: 0.2775 - acc: 0.9066 - val_loss: 0.2424 - val_acc: 0.9178
Epoch 59/100
110288/110288 [==============================] - 22s - loss: 0.2811 - acc: 0.9058 - val_loss: 0.2534 - val_acc: 0.9154
Epoch 60/100
110288/110288 [==============================] - 22s - loss: 0.2806 - acc: 0

### This cell can be used to load the model

In [11]:
# Loading the model
embed_rnn_model = load_model(embed_rnn_file)

# Preprocess inputs
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])

# Print predictions
print_predictions(model=embed_rnn_model, X=tmp_x,
                  y=french_sentences, tokenizer=french_tokenizer, number_of_predictions=5)

-------------------------------------------------------

Predicted sentence:
new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct translation:
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
-------------------------------------------------------

Predicted sentence:
les états unis est généralement froid en juillet et il est généralement en l' <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct translation:
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
-------------------------------------------------------

Predicted sentence:
californie est généralement calme en mois et il est généralement chaud en juin <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct translation:
california est généralement calme en mars , et il est généralement chaud en juin .
-------------------------------------------------------

Predicted sentence:
les états unis

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [19]:
from keras.layers import Bidirectional


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate = 0.01
    
    inputs = Input(shape=input_shape[1:])
    gru0 = Bidirectional(GRU(14, return_sequences=True))(inputs)
    gru1 = Bidirectional(GRU(28, return_sequences=True))(gru0)
    gru2 = Bidirectional(GRU(56, return_sequences=True))(gru1)
    dense0 = TimeDistributed(Dense(english_vocab_size + french_vocab_size, activation='tanh'))(gru2)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dense0)
    
    model = Model(inputs=inputs, outputs=logits)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[1], 1))

bd_rnn_model = bd_model(input_shape=tmp_x.shape,
                        output_sequence_length=preproc_english_sentences.shape[1],
                        english_vocab_size=len(english_tokenizer.word_index) + 1,
                        french_vocab_size=len(french_tokenizer.word_index) + 1)

bd_rnn_model.summary()

checkpointer = ModelCheckpoint(filepath=bd_rnn_best_weights, save_best_only=True, verbose=1)

bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024,
                 epochs=60, validation_split=0.2,
                 callbacks=[checkpointer])

# Loading best weights and saving the model
print()
print('Loading the best weights')
bd_rnn_model.load_weights(bd_rnn_best_weights)
print()
print('Saving the model for later usage...')
bd_rnn_model.save(bd_rnn_file)

# Printing and comparing predictions
print_predictions(model=bd_rnn_model, X=tmp_x,
                  y=french_sentences, tokenizer=french_tokenizer, number_of_predictions=5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 21, 28)            1344      
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 21, 56)            9576      
_________________________________________________________________
bidirectional_24 (Bidirectio (None, 21, 112)           37968     
_________________________________________________________________
time_distributed_23 (TimeDis (None, 21, 545)           61585     
_________________________________________________________________
time_distributed_24 (TimeDis (None, 21, 345)           188370    
Total params: 298,843
Trainable params: 298,843
Non-trainable params: 0
_________________________________________________________________
Trai

110288/110288 [==============================] - 24s - loss: 0.4971 - acc: 0.8206 - val_loss: 0.4141 - val_acc: 0.8531
Epoch 26/60
110288/110288 [==============================] - 24s - loss: 0.4926 - acc: 0.8228 - val_loss: 0.4349 - val_acc: 0.8384
Epoch 27/60
110288/110288 [==============================] - 24s - loss: 0.4853 - acc: 0.8308 - val_loss: 0.4407 - val_acc: 0.8410
Epoch 28/60
110288/110288 [==============================] - 24s - loss: 0.4649 - acc: 0.8347 - val_loss: 0.3851 - val_acc: 0.8649
Epoch 29/60
110288/110288 [==============================] - 24s - loss: 0.5710 - acc: 0.8072 - val_loss: 0.6265 - val_acc: 0.7825
Epoch 30/60
110288/110288 [==============================] - 24s - loss: 0.4732 - acc: 0.8289 - val_loss: 0.5799 - val_acc: 0.7939
Epoch 31/60
110288/110288 [==============================] - 24s - loss: 0.5618 - acc: 0.8061 - val_loss: 0.4401 - val_acc: 0.8468
Epoch 32/60
110288/110288 [==============================] - 24s - loss: 0.4449 - acc: 0.8421 -

### This cell can be used to load the model and print some predictions

In [12]:
# Loading the model
bd_rnn_model = load_model(bd_rnn_file)

# Preprocessing inputs
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[1], 1))

# Print predictions
print_predictions(model=bd_rnn_model, X=tmp_x,
                  y=french_sentences, tokenizer=french_tokenizer, number_of_predictions=5)

-------------------------------------------------------

Predicted sentence:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct translation:
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
-------------------------------------------------------

Predicted sentence:
les états unis est généralement froid en juillet et il gèle habituellement en novembre <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct translation:
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
-------------------------------------------------------

Predicted sentence:
california est généralement calme en mars et il est généralement chaud en juin <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct translation:
california est généralement calme en mars , et il est généralement chaud en juin .
-------------------------------------------------------

Predicted sentence:


### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [45]:
from keras.layers import RepeatVector


def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    learning_rate = 0.01
    
    inputs = Input(shape=input_shape[1:])
    
    # Encoder part
    enc_gru0 = GRU(200, dropout=0.2)(inputs)
    
    # Decoder part
    repeat = RepeatVector(output_sequence_length)(enc_gru0)
    dec_gru0 = GRU(200, return_sequences=True, dropout=0.2)(repeat)
    
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dec_gru0)
    
    model = Model(inputs=inputs, outputs=logits)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[1], 1))

encdec_rnn_model = encdec_model(tmp_x.shape,
                                preproc_french_sentences.shape[1],
                                len(english_tokenizer.word_index.keys()) + 1,
                                len(french_tokenizer.word_index.keys()) + 1)

encdec_rnn_model.summary()

# Training the model
checkpointer = ModelCheckpoint(filepath=encdec_rnn_best_weights, save_best_only=True, verbose=1)

encdec_rnn_model.fit(tmp_x,
                     preproc_french_sentences,
                     batch_size=1024, epochs=15, validation_split=0.25,
                     callbacks=[checkpointer])

# Loading best weights
print('Loading best weights...')
encdec_rnn_model.load_weights(encdec_rnn_best_weights)

# Saving model for later use
print('Saving model...')
encdec_rnn_model.save(encdec_rnn_file)

# Printing and comparing predictions
print_predictions(model=encdec_rnn_model, X=tmp_x,
                  y=french_sentences, tokenizer=french_tokenizer, number_of_predictions=5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
gru_71 (GRU)                 (None, 200)               121200    
_________________________________________________________________
repeat_vector_23 (RepeatVect (None, 21, 200)           0         
_________________________________________________________________
gru_72 (GRU)                 (None, 21, 200)           240600    
_________________________________________________________________
time_distributed_46 (TimeDis (None, 21, 345)           69345     
Total params: 431,145
Trainable params: 431,145
Non-trainable params: 0
_________________________________________________________________
Train on 103395 samples, validate on 34466 samples
Epoch 1/15
103395/103395 [==============================] - 22s - loss: 2.7374 - acc:

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [17]:
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout

def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate = 0.001
    
    # ------- INPUT LAYER -------
    inputs =  Input(shape=input_shape[1:])
    # ------- ERMBEDDING LAYER -------
    embed = Embedding(input_dim=english_vocab_size, output_dim=60)(inputs)
    # ------- ENCODER GRU LAYER 0 -------
    enc_gru0 = Bidirectional(GRU(200, dropout=0.35))(embed)
    # ------- REPEAT LAYER -------
    repeat = RepeatVector(output_sequence_length)(enc_gru0)
    # ------- DECODER GRU LAYER 0 -------
    dec_gru0 = Bidirectional(GRU(200, return_sequences=True, dropout=0.35))(repeat)
    # ------- DENSE LAYER 0 -------
    dense1 = TimeDistributed(Dense(256, activation='relu'))(dec_gru0)
    dense1 = Dropout(0.1)(dense1)
    # ------- OUTPUT LAYER -------
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dense1)
    
    model = Model(inputs=inputs, outputs=logits)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_model_final(model_final)


print('Final Model Loaded')

Final Model Loaded


### Training the final model

In [19]:
final_rnn_model = model_final(preproc_english_sentences.shape,
                              preproc_french_sentences.shape[1],
                              len(english_tokenizer.word_index.keys()) + 1,
                              len(french_tokenizer.word_index.keys()) + 1)

final_rnn_model.summary()

checkpointer =  ModelCheckpoint(final_rnn_best_weights, save_best_only=True, verbose=2)

final_rnn_model.fit(preproc_english_sentences,
                    preproc_french_sentences,
                    batch_size=1024, epochs=45, validation_split=0.25,
                    callbacks=[checkpointer])

# Loading the best weigts
print('Loading best weights...')
final_rnn_model.load_weights(final_rnn_best_weights)

# Saving the model
print('Saving the model for later use')
final_rnn_model.save(final_rnn_file)

# Printing some results
print_predictions(model=final_rnn_model, X=preproc_english_sentences,
                  y=french_sentences, tokenizer=french_tokenizer, number_of_predictions=5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 15)                0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 15, 60)            12000     
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 400)               313200    
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 21, 400)           0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 21, 400)           721200    
_________________________________________________________________
time_distributed_13 (TimeDis (None, 21, 256)           102656    
_________________________________________________________________
dropout_7 (Dropout)          (None, 21, 256)           0         
__________

103395/103395 [==============================] - 35s - loss: 0.4214 - acc: 0.8682 - val_loss: 0.3249 - val_acc: 0.9047
Epoch 23/45
103395/103395 [==============================] - 35s - loss: 0.3907 - acc: 0.8774 - val_loss: 0.2979 - val_acc: 0.9123
Epoch 24/45
103395/103395 [==============================] - 35s - loss: 0.3585 - acc: 0.8874 - val_loss: 0.2622 - val_acc: 0.9228
Epoch 25/45
103395/103395 [==============================] - 35s - loss: 0.3348 - acc: 0.8945 - val_loss: 0.2606 - val_acc: 0.9222
Epoch 26/45
103395/103395 [==============================] - 35s - loss: 0.3117 - acc: 0.9020 - val_loss: 0.2252 - val_acc: 0.9345
Epoch 27/45
103395/103395 [==============================] - 35s - loss: 0.2936 - acc: 0.9080 - val_loss: 0.2226 - val_acc: 0.9329
Epoch 28/45
103395/103395 [==============================] - 35s - loss: 0.2778 - acc: 0.9130 - val_loss: 0.1979 - val_acc: 0.9417
Epoch 29/45
103395/103395 [==============================] - 35s - loss: 0.2570 - acc: 0.9195 -

## Prediction (IMPLEMENTATION)

In [20]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = load_model(final_rnn_file)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.